In [35]:
import numpy as np
from fractions import Fraction
import pandas as pd

In [196]:
def make_coeff_table(n_x_vars, equations, function):
    """
    По заданным ограничениям строим симплекс-таблицу
    """
    
    n_s_vars = 0
    total_vars = n_x_vars
    
    # добавление слабых переменных в ограничения
    for equation in equations:
        if '>=' in equation or '<=' in equation:
            n_s_vars += 1
            total_vars += 1
        
    coeff_table = [[Fraction("0/1") for i in range(total_vars+1)] for j in range(len(equations)+1)]        
    s_index = n_x_vars # индекс, начиная с которого будут располагаться значения s в матрице
    
    for i in range(1, len(equations)+1):
        equation = equations[i - 1].split(' ') # разбиваем ограничение по пробелам, чтобы уметь доставать любую переменную, коэффициент или операцию

        for j in range(len(equation)):
            if 'x_' in equation[j]:
                coeff, index = equation[j].split('x_') # т.к. переменные у нас вида ax_i, то разделяя по x_, вытаскиваем коэффициент и индекс переменной
 
                if equation[j - 1] == '-':
                    coeff_table[i][int(index) - 1] = Fraction("-" + coeff + "/1")
                else:
                    coeff_table[i][int(index) - 1] = Fraction(coeff + "/1")
            
            elif equation[j] == '<=':
                coeff_table[i][s_index] = Fraction("1/1")
                s_index += 1
                
            elif equation[j] == '>=':
                coeff_table[i][s_index] = Fraction("-1/1") 
                s_index += 1
                
            if j == len(equation) - 1:
                if equation[j - 1] == '-':
                    coeff_table[i][total_vars] = Fraction("-" + equation[j] + "/1")
                else:
                    coeff_table[i][total_vars] = Fraction(equation[j] + "/1")
    
    
    goal_function = function[0].split(' ')
            
    for j in range(len(goal_function)):
        if 'x_' in goal_function[j]:
            if function[1] == 'max':
                coeff, index = goal_function[j].split('x_')
                
                if goal_function[j - 1] == '-':
                    coeff_table[0][int(index) - 1] = Fraction(coeff + "/1")
                else:
                    coeff_table[0][int(index) - 1] = Fraction("-" + coeff + "/1")
            else:
                coeff, index = goal_function[j].split('x_')
                
                if goal_function[j - 1] == '-':
                    coeff_table[0][int(index) - 1] = Fraction("-" + coeff + "/1")
                else:
                    coeff_table[0][int(index) - 1] = Fraction(coeff + "/1")
                    
                    
    columns = []
    for i in range(n_x_vars):
        columns.append(f"x_{i+1}")
    
    for i in range(n_s_vars):
        columns.append(f"s_{i+1}")
    
    columns.append('basic')
    
    coeff_table = pd.DataFrame(coeff_table, columns=columns)
                            
    return coeff_table, columns


def Gauss(coeff_table, lead_elem, columns):
    pass


def unlimited_check(coeff_table, lead_column):
    """
    Условие неограниченности задачи ЛП:
        Если в ведущем столбце s нет положительных коэффициентов,
        то значение задачи ЛП не ограничено (нет оптимального решения)
        
    Иначе:
        в качестве выводимой из базиса переменной x_r выбирается переменная,
        для которой:
            b_r / a_rs = min(b_i / a_is), a_is > 0
        r - ведущая строка (lead_row)
        a_rs - ведущий элемент
        
    Возвращает номер ведущей строки
    """
    fl = coeff_table[lead_column][1:].all() <= 0
    
    if fl:
        print("Нет оптимального решения")
    else:
        a_rs = np.array(coeff_table[lead_column][1:])
        b = np.array(coeff_table['basic'][1:])
        
        print(a_rs, b)
        
        res = []
        i = 0
        for a in a_rs:
            if a > 0:
                res.append(a / b[i])
                i += 1
        
        print(res)
        return min(res)


def optimality_check(coeff_table, columns):
    """
    Проверка условия оптимальности:
        Если все коэффициенты в нулевой строке симплексной таблицы,
        соответствующие целевой функции, неотрицательны, то текущее
        базисное решение оптимально
    
    Если существуют коэффициенты <0, то текущее базисное решение
    можно улучшить за счет введения в базис переменной, выбирающейся из условия: 
    c_s = min c_j; c_j<0
    
    Когда ведущий столбец найден, проверяем условие неограниченности и ищем ведущую строку
    Далее преобразовываем симлекс-таблицу с помощью метода Гаусса
    """
    fl = (coeff_table.iloc[0]).all() > 0
    
    while not fl:
        row_0 = np.array(coeff_table[coeff_table.columns[:-1]].loc[0])
        
        min_c = row_0.min()
        lead_column = row_0.argmin()
        lead_column = columns[lead_column]
        
        if fl:
            return coeff_table
        else:
            lead_row = unlimited_check(coeff_table, lead_column)
            lead_elem = (lead_row, lead_column)
            coeff_table = Gauss(coeff_table, lead_elem, columns)
    
     # lead_elem = coeff_table[row_index][column_index]
    return coeff_table
    

def simplex(n_x_vars, equations, function):
    """
    :param vars: Число переменных в задаче ЛП

    :param equations: Список элементов типа string со всеми ограничениями:
                        1. правила для переменной:
                            записана с коэффициентом
                            через "_" должен быть прописан ее номер
                        2. используются только одиночные пробелы, между коэффициентом
                            и переменной пробела нет
                        Пример:
                            ['1x_1 + 7x_2 + 2x_3 <= 9', '2x_2 + 4x_1 >= 5', 'x_3 + 2x_2 = 3']

    :param function: Тип list с двумя параметрами:
                            1. min или max
                            2. функция, с которой работаем
    """
    coeff_table, columns = make_coeff_table(n_x_vars, equations, function)
    
    print(coeff_table)
    print(optimality_check(coeff_table, columns))

In [197]:
def input_file():
    print("Для корректного ввода прочитайте правила оформления в файле README.md")
    print("Введите название файла")
    file = input()
    
    f = open(file, "r", encoding='utf-8')
    equations = []
    k = 0
    for line in f:
        if k == 0:
            n_x_vars = int(line.replace('\n', ''))
        elif k == 1:
            function = line.replace('\n', '')
        else:
            equations.append(line.replace('\n', ''))
        k += 1
    
    function = function.split(', ')
    
    return function, equations, n_x_vars
    

def input_console():
    print("Для корректного ввода прочитайте правила оформления в файле README.md")
    print("Введите функцию и цель исследования - min/max - через запятую и пробел после нее")
    function = input().split(', ')
    
    print("Введите количество переменных:")
    n_x_vars = int(input())
    
    print("Введите количество ограничений:")
    n = int(input())
    
    print(f"На каждой строке через enter введите {n} ограничений:")
    equations = []
    for i in range(n):
        equations.append(input())
        
    return function, equations, n_x_vars

In [198]:
if __name__ == '__main__':
    print("Укажите как хотите вводить исходные данные: консоль/файл")
    type_input = input().lower()
    
    assert type_input in ['консоль', 'console', 'file', 'файл'], 'Такой тип не поддерживается'
    
    if type_input in ['консоль', 'console']:
        function, equations, n_x_vars = input_console()
    
    elif type_input in ['file', 'файл']:
        function, equations, n_x_vars = input_file()
    
    simplex(n_x_vars, equations, function)

Укажите как хотите вводить исходные данные: консоль/файл
file
Для корректного ввода прочитайте правила оформления в файле README.md
Введите название файла
text.txt
  x_1 x_2 s_1 s_2 s_3 s_4 basic
0  -1  -3   0   0   0   0     0
1   3   2   1   0   0   0    12
2   1   2   0   1   0   0     6
3  -1   1   0   0   1   0     1
4   0   1   0   0   0   1     2
[Fraction(2, 1) Fraction(2, 1) Fraction(1, 1) Fraction(1, 1)] [Fraction(12, 1) Fraction(6, 1) Fraction(1, 1) Fraction(2, 1)]
[Fraction(1, 6), Fraction(1, 3), Fraction(1, 1), Fraction(1, 2)]


NameError: name 'lead_columns' is not defined

In [ ]:
file
te